# Insertion of HemA and HemL

To enable our organism to make heme from L-glumamate (rather than just glycine), we add new reactions to the model to simulate the heterologous expression of the genes for HemA and HemL, which are found in plants and bacteria.

First, packages and the model is imported. Additionally, a copy of the model is created to enable us to compare the two models later.

In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [2]:
model = read_sbml_model('model_yeast8_rhb.xml')

In [3]:
model_base = model.copy()

The "last" metabolite made by the native yeast metabolism, which will be our starting point for the HemA reaction is L-Glutamyl-tRNA(Glu) (id: glutrna_c). This metabolite is made from L-glutamate.

In [4]:
model.metabolites.get_by_id('glutrna_c')

Metabolite identifier,glutrna_c
Name,Glu-tRNA(Glu)
Memory address,0x02247ffb4190
Formula,C5H8NO3R
Compartment,c
In 2 reaction(s),"BIOMASS_yeast-GEM_PROT, GLUTRS"


The metabolite where the plant heme biosynthesis "re-connects" with the native yeast metabolism is 5-Amino-4-oxopentanoate / 5-Aminolevulinate (id: 5aop_c). 

In [5]:
model.metabolites.get_by_id('5aop_c')

Metabolite identifier,5aop_c
Name,5-aminolevulinate
Memory address,0x02247ff1eee0
Formula,C5H9NO3
Compartment,c
In 3 reaction(s),"5AOPt2, PPBNGS, 5AOPtm"


We are now interested in adding two reactions, HemA and HemL, which connects the two metabolites mentioned above, thereby enabling the yeast to utilize L-glumamate. The reaction formulas are:

HemA (glutamyl-tRNA reductase, https://www.kegg.jp/entry/1.2.1.70): 
L-glutamyl-tRNAGlu + NADPH + H+ -> L-glutamate 1-semialdehyde + NADP+ + tRNAGlu

HemL (glutamate-1-semialdehyde aminotransferase, https://www.kegg.jp/entry/5.4.3.8): L-glutamate 1-semialdehyde -> 5-aminolevulinate

Note that it is unknown whether the reactions would be active in the cytoplasm or mitochondria when expressed in *Saccharomyces cerevisiae*. Therefore, it is assumed for simplicity that the reactions would be active in the cytoplasm.

First the new intermediate formed by HemA (and consumed by HemL) is defined (L-glutamate 1-semialdehyde):

In [6]:
L_glu_1_semi = Metabolite(id='L_glu_1_semi_c', compartment='c')

The HemA reaction is defined:

In [7]:
HemA = Reaction('HemA')

In [8]:
HemA.add_metabolites({model.metabolites.h_c: -1,
                      model.metabolites.nadph_c: -1,
                      model.metabolites.glutrna_c: -1,
                      L_glu_1_semi: 1,
                      model.metabolites.nadp_c: 1,
                      model.metabolites.trnaglu_c: 1,
                             })

In [9]:
print(HemA.build_reaction_string())

glutrna_c + h_c + nadph_c --> L_glu_1_semi_c + nadp_c + trnaglu_c


HemL is defined:

In [10]:
HemL = Reaction('HemL')

In [11]:
HemL.add_metabolites({L_glu_1_semi: -1,
                      model.metabolites.get_by_id('5aop_c'): 1,
                             })

In [12]:
print(HemL.build_reaction_string())

L_glu_1_semi_c --> 5aop_c


Both reactions are added to the model:

In [13]:
model.add_reactions([HemA, HemL])

In [14]:
import escher

The new pathway is visualized in red below in addition to the native heme synthesis pathway, which ultimately leads to hemoglobin production.

In [15]:
escher.Builder(model = model,
    map_json='Yeast8_hemA_hemL.json',
              reaction_data={'HemA': 10, 'HemL': 10})

Builder(reaction_data={'HemA': 10, 'HemL': 10})

## Investigation of the effect of inserting HemA and HemL

In [16]:
# Mutant with HemA and HemL:
with model:
    model.objective = model.reactions.EX_rHb
    model.optimize()
    print('Parameters for the HemA and HemL strain:')
    print("Rate of heme production:", model.reactions.EX_rHb.flux)
    print("Growth rate:", model.reactions.GROWTH.flux)
    print("Yield of heme production:", model.reactions.EX_rHb.flux / (-1*model.reactions.EX_glc__D_e.flux))

Parameters for the HemA and HemL strain:
Rate of heme production: 0.01633390069511922
Growth rate: 0.0
Yield of heme production: 0.001633390069511922


In [17]:
# Non-mutant yeast
with model_base:
    model_base.objective = model_base.reactions.EX_rHb
    model_base.optimize()
    print('Parameters for the WT strain without HemA and HemL:')
    print("Rate of heme production:", model_base.reactions.EX_rHb.flux)
    print("Growth rate:", model_base.reactions.GROWTH.flux)
    print("Yield of heme production:", model_base.reactions.EX_rHb.flux / (-1*model.reactions.EX_glc__D_e.flux))

Parameters for the WT strain without HemA and HemL:
Rate of heme production: 0.016333900695119207
Growth rate: 0.0
Yield of heme production: 0.0016333900695119207


From the above, it looks like the HemA and HemL genes does not give the organism an added advantage under normal conditions when compared to the WT. However, there could perhaps be situations in which it is advantageous to have several routes for the flux to go. 